In [18]:
import pandas as pd
import numpy as np
import math 
import tensorflow as tf
df = pd.read_csv("../../data/raw/news/newsapiorg_labelled.csv", index_col=0)
df['label'] = df['label']+1
df.head()

,title,publishedAt,url,content,vader_neg,vader_neu,vader_pos,vader_comp,vader_label,label,include,relevant
Column1,,,,,,,,,,,,
1,Key remarks from Russian c.bank governor after...,2022-02-11T12:26:00Z,https://www.reuters.com/markets/currencies/key...,"MOSCOW, Feb 11 (Reuters) - The Russian Central...",0.000,1.000,0.0,0.0000,0,1,1,0
2,LIVE MARKETS Forex: worrying about an inverted...,2022-02-18T12:33:00Z,https://www.reuters.com/markets/stocks/live-ma...,Feb 18 - Welcome to the home for real-time cov...,0.211,0.789,0.0,-0.3400,-1,0,1,1
3,Dollar takes a break from this week's slide ah...,2022-02-03T02:18:00Z,https://www.reuters.com/markets/europe/dollar-...,"SINGAPORE, Feb 3 (Reuters) - The dollar found ...",0.000,1.000,0.0,0.0000,0,0,1,1
4,Euro bounce pauses ahead of U.S. inflation - R...,2022-02-08T01:16:00Z,https://www.reuters.com/business/euro-bounce-p...,"SINGAPORE, Feb 8 (Reuters) - A resurgent euro ...",0.000,1.000,0.0,0.0000,0,2,1,1
5,Dollar near two-week high amid jitters over ha...,2022-01-25T01:52:00Z,https://www.reuters.com/markets/europe/dollar-...,"TOKYO, Jan 25 (Reuters) - The safe-haven U.S. ...",0.197,0.803,0.0,-0.4019,-1,0,1,1


In [19]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [12]:
import sklearn
from sklearn.model_selection import train_test_split
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(df.title, df.label, test_size = 0.2)
train = pd.concat([train_inputs, train_labels], axis = 1)
test = pd.concat([valid_inputs, valid_labels], axis = 1)
train.head()

,title,label
Column1,,
324,"Bloomberg Businessweek Weekend - January 28th,...",1
1463,Suominen Corporation’s Financial Statements Re...,2
50,European stocks join Asia sell-off as Russia-U...,0
782,Reflections on the 2022 FCT Area Council Elect...,1
1603,LIVE MARKETS STOXX near 2-week high - Reuters,2


In [13]:
def convert_example_to_feature(text): 
    return tokenizer.encode_plus(text,
        add_special_tokens = True,
        max_length = 512,
        pad_to_max_length = True,
        return_attention_mask = True,
    )

def map_example_to_dict(input_ids, attention_masks, token_type_ids, label): 
    return {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_masks,
    },label

In [28]:
def encode(df):
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    for index,row in df.iterrows():
        bert_input = convert_example_to_feature(row['title'])
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(row['label'])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


In [29]:
ds_train_encoded = encode(train).shuffle(100).batch(16)
ds_test_encoded = encode(test).batch(16)

C:\Users\wangt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [30]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf
from tensorflow import keras
from keras import layers

learning_rate = 2e-5
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
bert_history = model.fit(ds_train_encoded, epochs=5, validation_data=ds_test_encoded)

Epoch 1/5
108/108 [==============================] - 5789s 54s/step - loss: 0.8563 - accuracy: 0.6092 - val_loss: 0.6408 - val_accuracy: 0.7494
Epoch 2/5
108/108 [==============================] - 31179s 291s/step - loss: 0.5369 - accuracy: 0.7985 - val_loss: 0.5781 - val_accuracy: 0.7958
Epoch 3/5
108/108 [==============================] - 5149s 48s/step - loss: 0.3858 - accuracy: 0.8693 - val_loss: 0.6142 - val_accuracy: 0.7819
Epoch 4/5
108/108 [==============================] - 5033s 47s/step - loss: 0.2756 - accuracy: 0.9193 - val_loss: 0.6352 - val_accuracy: 0.7935
Epoch 5/5
108/108 [==============================] - 5009s 46s/step - loss: 0.2014 - accuracy: 0.9355 - val_loss: 0.7155 - val_accuracy: 0.7773


In [33]:
from joblib import dump
import os
MODEL_PATH = "../../results/models/BERT_v1.0"
model.save(MODEL_PATH)
dump(bert_history, os.path.join(MODEL_PATH, "bert_history.save"))

INFO:tensorflow:Assets written to: ../../results/models/BERT_v1.0\assets


INFO:tensorflow:Assets written to: ../../results/models/BERT_v1.0\assets


INFO:tensorflow:Assets written to: ram://ccc60962-787a-445d-9763-58c50c516ff6/assets


INFO:tensorflow:Assets written to: ram://ccc60962-787a-445d-9763-58c50c516ff6/assets


['../../results/models/BERT_v1.0\\bert_history.save']

In [38]:
from sklearn.metrics import accuracy_score
from joblib import load
MODEL_PATH = "../../results/models/BERT_v1.0"
loaded_bert = tf.keras.models.load_model(MODEL_PATH)